# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Sep 19 2022 10:18AM,246696,19,8546278,"Graystone, LLC",Released
1,Sep 19 2022 10:18AM,246696,19,8546289,"Graystone, LLC",Released
2,Sep 19 2022 10:08AM,246694,10,MSP214035,"Methapharm, Inc.",Released
3,Sep 19 2022 10:08AM,246694,10,MSP214036,"Methapharm, Inc.",Released
4,Sep 19 2022 10:08AM,246694,10,MSP214039,"Methapharm, Inc.",Released
5,Sep 19 2022 10:08AM,246694,10,MSP214042,"Methapharm, Inc.",Released
6,Sep 19 2022 10:08AM,246694,10,MSP214043,"Methapharm, Inc.",Released
7,Sep 19 2022 10:08AM,246694,10,MSP214044,"Methapharm, Inc.",Released
8,Sep 19 2022 10:08AM,246694,10,MSP214045,"Methapharm, Inc.",Released
9,Sep 19 2022 10:08AM,246694,10,MSP214046,"Methapharm, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
9,246691,Released,15
10,246693,Released,1
11,246694,Released,10
12,246695,Released,3
13,246696,Released,2


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
246691,26.0,15.0
246693,NaN,1.0
246694,NaN,10.0
246695,NaN,3.0
246696,NaN,2.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
246672,6.0,1.0
246677,1.0,0.0
246678,5.0,5.0
246680,1.0,0.0
246684,0.0,48.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
246672,6,1
246677,1,0
246678,5,5
246680,1,0
246684,0,48


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,246672,6,1
1,246677,1,0
2,246678,5,5
3,246680,1,0
4,246684,0,48


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,246672,6,1
1,246677,1,
2,246678,5,5
3,246680,1,
4,246684,,48


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Sep 19 2022 10:18AM,246696,19,"Graystone, LLC"
2,Sep 19 2022 10:08AM,246694,10,"Methapharm, Inc."
12,Sep 19 2022 10:06AM,246695,10,"Methapharm, Inc."
15,Sep 19 2022 9:56AM,246693,16,Sartorius Stedim North America
16,Sep 19 2022 9:36AM,246677,16,"Emersa Waterbox, LLC"
17,Sep 19 2022 9:28AM,246691,12,Hush Hush
58,Sep 19 2022 9:17AM,246688,10,ISDIN Corporation
78,Sep 19 2022 8:49AM,246684,10,ISDIN Corporation
126,Sep 19 2022 8:25AM,246680,16,Sartorius Stedim North America
127,Sep 19 2022 8:16AM,246678,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Sep 19 2022 10:18AM,246696,19,"Graystone, LLC",,2
1,Sep 19 2022 10:08AM,246694,10,"Methapharm, Inc.",,10
2,Sep 19 2022 10:06AM,246695,10,"Methapharm, Inc.",,3
3,Sep 19 2022 9:56AM,246693,16,Sartorius Stedim North America,,1
4,Sep 19 2022 9:36AM,246677,16,"Emersa Waterbox, LLC",1,
5,Sep 19 2022 9:28AM,246691,12,Hush Hush,26,15
6,Sep 19 2022 9:17AM,246688,10,ISDIN Corporation,,20
7,Sep 19 2022 8:49AM,246684,10,ISDIN Corporation,,48
8,Sep 19 2022 8:25AM,246680,16,Sartorius Stedim North America,1,
9,Sep 19 2022 8:16AM,246678,10,ISDIN Corporation,5,5


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Sep 19 2022 10:18AM,246696,19,"Graystone, LLC",2,
1,Sep 19 2022 10:08AM,246694,10,"Methapharm, Inc.",10,
2,Sep 19 2022 10:06AM,246695,10,"Methapharm, Inc.",3,
3,Sep 19 2022 9:56AM,246693,16,Sartorius Stedim North America,1,
4,Sep 19 2022 9:36AM,246677,16,"Emersa Waterbox, LLC",,1
5,Sep 19 2022 9:28AM,246691,12,Hush Hush,15,26
6,Sep 19 2022 9:17AM,246688,10,ISDIN Corporation,20,
7,Sep 19 2022 8:49AM,246684,10,ISDIN Corporation,48,
8,Sep 19 2022 8:25AM,246680,16,Sartorius Stedim North America,,1
9,Sep 19 2022 8:16AM,246678,10,ISDIN Corporation,5,5


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Sep 19 2022 10:18AM,246696,19,"Graystone, LLC",2,
1,Sep 19 2022 10:08AM,246694,10,"Methapharm, Inc.",10,
2,Sep 19 2022 10:06AM,246695,10,"Methapharm, Inc.",3,
3,Sep 19 2022 9:56AM,246693,16,Sartorius Stedim North America,1,
4,Sep 19 2022 9:36AM,246677,16,"Emersa Waterbox, LLC",,1


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Sep 19 2022 10:18AM,246696,19,"Graystone, LLC",2.0,NaN
1,Sep 19 2022 10:08AM,246694,10,"Methapharm, Inc.",10.0,NaN
2,Sep 19 2022 10:06AM,246695,10,"Methapharm, Inc.",3.0,NaN
3,Sep 19 2022 9:56AM,246693,16,Sartorius Stedim North America,1.0,NaN
4,Sep 19 2022 9:36AM,246677,16,"Emersa Waterbox, LLC",NaN,1.0


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Sep 19 2022 10:18AM,246696,19,"Graystone, LLC",2.0,0.0
1,Sep 19 2022 10:08AM,246694,10,"Methapharm, Inc.",10.0,0.0
2,Sep 19 2022 10:06AM,246695,10,"Methapharm, Inc.",3.0,0.0
3,Sep 19 2022 9:56AM,246693,16,Sartorius Stedim North America,1.0,0.0
4,Sep 19 2022 9:36AM,246677,16,"Emersa Waterbox, LLC",0.0,1.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,1233439,86.0,5.0
12,246691,15.0,26.0
16,740050,1.0,2.0
19,493368,3.0,6.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,1233439,86.0,5.0
1,12,246691,15.0,26.0
2,16,740050,1.0,2.0
3,19,493368,3.0,6.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,86.0,5.0
1,12,15.0,26.0
2,16,1.0,2.0
3,19,3.0,6.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,86.0
1,12,Released,15.0
2,16,Released,1.0
3,19,Released,3.0
4,10,Executing,5.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,16,19
Status,,,,
Executing,5.0,26.0,2.0,6.0
Released,86.0,15.0,1.0,3.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,16,19
0,Executing,5.0,26.0,2.0,6.0
1,Released,86.0,15.0,1.0,3.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,16,19
0,Executing,5.0,26.0,2.0,6.0
1,Released,86.0,15.0,1.0,3.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()